In [5]:
from langchain_community.llms import Ollama
import time

In [77]:
model = Ollama(model="gemma2")
def run(chain,prompt=""): 
    tokens = ""
    start = time.time()
    for chunk in chain.stream(prompt, ): 
        print(chunk, end='', flush=True)
        tokens += chunk
    end = time.time()
    print(f"\n\n => tokens/second : {len(tokens.split(" "))/(end-start)}")

In [78]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableBranch

In [79]:
prompt = PromptTemplate.from_template(
    """
    You are a Calssifer assistant that helps me 
    classify customers feedback as Good, Bad, Neutral or Don't know 
    please only stick to these options and only output one word 
    nothing more nothing less 
    I want you to classify this : 
    {feedback}
    """
)
parser = StrOutputParser()

classifier = prompt | model | parser

In [80]:
good_chain = (lambda x: "write a possitive message with emoji's") | model | parser
bad_chain = (lambda x: "write a negative message with emoji's") | model | parser
neutral_chain = (lambda x: "write a Neutral message with emoji's") | model | parser
idk_chain = (lambda x: "write only random emoji's without saying anythin") | model | parser

In [85]:
router_chain = classifier | RunnableBranch(
    (
        lambda x: "Good" in x,
        good_chain
    ),
    (
        lambda x: "Bad" in x,
        bad_chain
    ),
    (
        lambda x: "Neutral" in x,
        neutral_chain
    ),
    idk_chain
)

In [89]:
run(router_chain, "look I don't think its good or bad")

👋 Just wanted to say hello and see how you're doing! 😊  


 => tokens/second : 2.664875689219304
